#  PyTorch Lightning. Разворачивание веб-сервера для использования моделей.

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* https://lightning.ai/docs/pytorch/stable/starter/introduction.html
* https://lightning.ai/docs/pytorch/stable/levels/core_skills.html
* https://lightning.ai/docs/pytorch/stable/api/lightning.pytorch.core.LightningModule.html#lightning.pytorch.core.LightningModule.log
* https://lightning.ai/docs/pytorch/stable/extensions/logging.html
* https://lightning.ai/docs/pytorch/stable/common/progress_bar.html
* https://lightning.ai/docs/pytorch/stable/common/early_stopping.html
* https://lightning.ai/docs/pytorch/1.6.3/api/pytorch_lightning.utilities.model_summary.html#pytorch_lightning.utilities.model_summary.ModelSummary
* https://torchmetrics.readthedocs.io/en/stable/pages/lightning.html
* https://pykit.org/how-to-run-python-flask-app-online-using-ngrok/

## Задачи для совместного разбора

1\. Создайте датасет для регрессии и обучите модель при помощи PyTorch Lightning.

In [1]:
import torch as th

In [ ]:
!pip install lightning

In [4]:
from sklearn.datasets import make_regression
from torch.utils.data import TensorDataset, DataLoader

In [5]:
X, y = make_regression(n_samples=256, n_features=10)
X = th.from_numpy(X).float()
y = th.from_numpy(y).float()
dataset = TensorDataset(X, y)
loader = DataLoader(dataset, batch_size=64, shuffle=True)

In [6]:
import lightning as L
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [9]:
class Model(L.LightningModule):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(10, 5)
    self.fc2 = nn.Linear(5, 1)

  def forward(self, X):
    out = self.fc1(X).relu()
    out = self.fc2(out)
    return out

  def configure_optimizers(self):
    optimizer = optim.Adam(self.parameters(), lr=0.1)
    return optimizer

  def training_step(self, batch, batch_idx):
    X, y = batch
    preds = self(X)
    loss = F.mse_loss(preds.flatten(), y)
    self.log('train_loss', loss, prog_bar=True)
    return loss

In [10]:
trainer = L.Trainer(max_epochs=50)
model = Model()
trainer.fit(model=model, train_dataloaders=loader)

INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: 
  | Name | Type   | Params
--------------------------------
0 | fc1  | Linear | 55    
1 | fc2  | Linear | 6     
--------------------------------
61        Trainable params
0         Non-trainable params
61        Total params
0.000     Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name | Type   | Params
--------------------------------
0 | fc1  | Linear | 55    
1 | fc2  | Linear | 6     
--------------------------------
61        Trainable params
0   

Training: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=50` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.


## Задачи для самостоятельного решения

<p class="task" id="1"></p>

1\. Опишите датасет `AnimalDetectionDataset` на основе архива `animals.zip`. Реализуйте `__getitem__` таким образом, чтобы он возвращал три элемента: тензор с изображением, словарь с координатами bounding box и метку объекта. Предусмотрите возможность передавать извне при создании датасета набор преобразований для изображений, преобразование для метки объекта (для кодирования) и флаг, показывающий, нужно ли возвращать исходные или нормированные координаты bounding box.  Разбейте набор данных на обучающее и валидационное множество. При создании датасета не забудьте указать преобразования, соответствующие модели ResNet.

- [ ] Проверено на семинаре

<p class="task" id="2"></p>

2\. Напишите модель для решения задачи выделения объектов в виде объекта `lightning.LightningModule`. Реализуйте двухголовую сеть, одна голова которой предсказывает метку объекта (задача классификации), а вторая голова предсказывает 4 координаты вершин bounding box (задача регрессии). В качестве backbone используйте модель resnet50 из пакета `torchvision`. В качестве функции потерь используйте сумму MSELoss (для расчета ошибки на задаче регрессии) и CrossEntropyLoss (для расчета ошибки на задачи классификации).

Реализуйте следующий функционал при помощи `lightning` и `torchmetrics`:
* для каждого батча во время обучения рассчитывается значение функции потерь и точности прогнозов, по завершению эпохи метрики усредняются;
* для каждого батча во время валидации рассчитывается значение функции потерь и точности прогнозов, по завершению эпохи метрики усредняются;
* если значение функции потерь не улучшалось в течении 5 эпох, происходит ранняя остановка;
* при создании модель на экран выводится сводка по модели с указанием размерностей выходов слоев;
* для визуализации процесса обучения используется tensorboard.

Используя обученную модель, получите предсказания для изображения кошки и собаки и отрисуйте их. Выполните процедуру, обратную нормализации, чтобы корректно отобразить фотографии.

- [ ] Проверено на семинаре

<p class="task" id="3"></p>

3\. Загрузите чекпоинт обученной модели и переведите модель в режим оценки. Допишите функцию `transform_image` и route `predict`. Запустите сервер flask и сделайте POST-запрос к соответствующему эндпоинту.

При работе в Google Colab вы можете воспользоваться инструментом `ngrok` для проброса локального адреса или запустить сервер Flask в отдельном потоке.

- [ ] Проверено на семинаре

In [ ]:
from PIL import Image
import io
from torchvision.transforms import v2 as T
import torch

def bytes_to_pil(image_bytes: bytes) -> Image:
    return Image.open(io.BytesIO(image_bytes))

def transform_image(image: Image) -> torch.Tensor:
    """Преобразует PIL.Image в тензор"""
    pass

In [ ]:
from flask import Flask
# from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify
import threading


ALLOWED_EXTENSIONS = {"png", "jpg", "jpeg"}


def allowed_file(filename):
    return "." in filename and filename.rsplit(".", 1)[1].lower() in ALLOWED_EXTENSIONS


app = Flask(__name__)  # app name
# run_with_ngrok(app)


@app.route("/predict", methods=["POST"])
def predict():
    if request.method == "POST":
        file = request.files.get("file")
        if file is None or file.filename == "":
            return jsonify({"error": "no file"})
        if not allowed_file(file.filename):
            return jsonify({"error": "format not supported"})
        try:
            img_bytes = file.read()
            image = bytes_to_pil(img_bytes)
            tensor = transform_image(image)

            # получите прогноз при помощи модели
            data = {
                "bbox": ...,
                "label": ...,
            }
            return jsonify(data)
        except Exception as e:
            return jsonify({"error": f"error during prediction: {e}"})


if __name__ == "__main__":
    threading.Thread(target=lambda: app.run()).start()
    # app.run()

In [ ]:
import requests

resp = requests.post(
    ".../predict",
    files={
        "file": open(
            "path/to/file", "rb"
        )
    },
)

print(resp.text)

## Обратная связь
- [ ] Хочу получить обратную связь по решению